In [ ]:
# Dataset
class MNISTDataset(Dataset):
    def __init__(self, mode='train', val_ratio=0.1, transform=None, root='./data', random_state=42):
        assert mode in ['train', 'val', 'test'], "mode must be 'train', 'val' or 'test'"
        self.mode = mode
        self.transform = transform

        # Temel dönüşüm
        base_transform = transforms.Compose([
            transforms.ToTensor()
        ])

        # MNIST veri setlerini yükle
        full_dataset = datasets.MNIST(root=root, train=True, download=True, transform=base_transform)
        test_dataset = datasets.MNIST(root=root, train=False, download=True, transform=base_transform)

        if mode == 'test':
            self.dataset = test_dataset
        else:
            all_indices = np.arange(len(full_dataset))
            all_labels = np.array(full_dataset.targets)

            # Stratified split
            train_idx, val_idx = train_test_split(
                all_indices,
                test_size=val_ratio,
                stratify=all_labels,
                random_state=random_state
            )

            self.dataset = Subset(full_dataset, train_idx if mode == 'train' else val_idx)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        x = x.view(-1)  # 28x28 → 784

        if self.transform:
            x_ = self.transform(x).to(torch.float32)
            return x, x_, y
        return x, y